In [ ]:
import pandas as pd
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio
import torch.nn.functional as F
from torch.nn import init
import torch
import torch.nn as nn
import csv
import tqdm

In [ ]:
### preparing dataset

def preprocess(file_path):
    df = pd.read_csv(file_path)

    print(df["action"].unique())
    print(df["object"].unique())
    print(df['Self-reported fluency level '].unique())
    df['fluency'] = df['Self-reported fluency level ']
    print(df['First Language spoken'].unique())
    df['FirstLanguage'] = df['First Language spoken']

    print(df['gender'].unique())
    print(df['Current language used for work/school'].unique())
    df['CurrentLanguage'] = df['Current language used for work/school']
    print(df['ageRange'].unique())

    df['actionObject'] = df["action"].astype(str) + df["object"].astype(str)

    df['actionObjectId'] = pd.factorize(df.actionObject)[0]
    df['actionId'] = pd.factorize(df.action)[0]
    df['objectId'] = pd.factorize(df.object)[0]
    df['fluencyId'] = pd.factorize(df.fluency)[0]
    df['firstLanguageId'] = pd.factorize(df.FirstLanguage)[0]
    df['genderId'] = pd.factorize(df.gender)[0]
    df['currentLanguageId'] = pd.factorize(df.CurrentLanguage)[0]
    df['ageRangeId'] = pd.factorize(df.ageRange)[0]


df = preprocess('/Users/Saeedeh/Downloads/dsl_data/development.csv')

['change language' 'activate' 'deactivate' 'increase' 'decrease']
['none' 'music' 'lights' 'volume' 'heat']
['advanced' 'native' 'basic' 'intermediate']
['English (United States)' 'French (Canada)' 'English (Canada)' 'Telugu'
 'Spanish (Venezuela)']
['female' 'male']
['English (United States)' 'English (Canada)' 'English (Australia)'
 'Spanish (Venezuela)']
['22-40' '41-65' '65+']


In [ ]:
df.head()

,Id,path,speakerId,action,object,Self-reported fluency level,First Language spoken,Current language used for work/school,gender,ageRange,...,CurrentLanguage,actionObject,actionObjectId,actionId,objectId,fluencyId,firstLanguageId,genderId,currentLanguageId,ageRangeId
0,0,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0a312...,2BqVo8kVB2Skwgyb,change language,none,advanced,English (United States),English (United States),female,22-40,...,English (United States),change languagenone,0,0,0,0,0,0,0,0
1,1,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0ee42...,2BqVo8kVB2Skwgyb,activate,music,advanced,English (United States),English (United States),female,22-40,...,English (United States),activatemusic,1,1,1,0,0,0,0,0
2,2,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/1d9f3...,2BqVo8kVB2Skwgyb,deactivate,lights,advanced,English (United States),English (United States),female,22-40,...,English (United States),deactivatelights,2,2,2,0,0,0,0,0
3,3,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/269fc...,2BqVo8kVB2Skwgyb,increase,volume,advanced,English (United States),English (United States),female,22-40,...,English (United States),increasevolume,3,3,3,0,0,0,0,0
4,4,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/5bbda...,2BqVo8kVB2Skwgyb,increase,volume,advanced,English (United States),English (United States),female,22-40,...,English (United States),increasevolume,3,3,3,0,0,0,0,0


In [ ]:

class AudioUtil():

  # Load an audio file
  @staticmethod
  def open(audio_file):
    sig, sr = torchaudio.load(audio_file)
    return (sig, sr)


  # Convert the given audio to the desired number of channels
  @staticmethod
  def rechannel(aud, new_channel):
    sig, sr = aud

    if (sig.shape[0] == new_channel):
      return aud

    if (new_channel == 1):
      resig = sig[:1, :]
    else:
      resig = torch.cat([sig, sig])

    return ((resig, sr))


  # Resample Signal
  @staticmethod
  def resample(aud, newsr):
    sig, sr = aud

    if (sr == newsr):
      return aud

    num_channels = sig.shape[0]
    resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1,:])
    if (num_channels > 1):
      retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:,:])
      resig = torch.cat([resig, retwo])

    return ((resig, newsr))


  # Pad (or truncate) the signal to a fixed length
  @staticmethod
  def pad_trunc(aud, max_ms):
    sig, sr = aud
    num_rows, sig_len = sig.shape
    max_len = sr//1000 * max_ms

    if (sig_len > max_len):
      # Truncate the signal to the given length
      sig = sig[:,:max_len]

    elif (sig_len < max_len):
      pad_begin_len = random.randint(0, max_len - sig_len)
      pad_end_len = max_len - sig_len - pad_begin_len 

      # Pad with 0s
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))

      sig = torch.cat(( pad_begin, sig, pad_end), 1)
      
    return (sig, sr)


  # Generate Spectrogram
  @staticmethod
  def spectrogram(aud, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud
    top_db = 80

    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    return (spec)


  # Generate MFCC
  @staticmethod
  def mfcc(aud, n_mfcc=64, n_mels=64, n_fft=1024, hop_len=None):
    sig,sr = aud

    mfcc = transforms.MFCC(sr, n_mfcc=n_mfcc, melkwargs={"n_fft": n_fft, "hop_length": hop_len, "n_mels": n_mels})(sig)
    return (mfcc)

In [ ]:

class SoundDS(Dataset):
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000 
    self.sr = 16000
    self.channel = 1
            
 
  def __len__(self):
    return len(self.df)    
    

  def __getitem__(self, idx):

    audio_file = self.data_path + self.df.loc[idx, 'path']

    labels = self.df.loc[idx, 'actionObjectId']
    fluencyId = self.df.loc[idx, 'fluencyId']
    firstLanguageId = self.df.loc[idx, 'firstLanguageId']
    genderId = self.df.loc[idx, 'genderId']
    currentLanguageId = self.df.loc[idx, 'currentLanguageId']
    ageRangeId = self.df.loc[idx, 'ageRangeId']

    #Preprocessing Signal
    aud = AudioUtil.open(audio_file)
    reaud = AudioUtil.resample(aud, self.sr)
    rechan = AudioUtil.rechannel(reaud, self.channel)
    dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
    sgram_mel = AudioUtil.spectrogram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    sgram_mfcc = AudioUtil.mfcc(dur_aud, n_mels=64, n_fft=1024, hop_len=None)

    # Concatenate spectrogram and mfcc
    sgram = torch.cat([sgram_mel, sgram_mfcc])

    return sgram, labels, fluencyId, firstLanguageId, genderId, currentLanguageId, ageRangeId

In [ ]:

sound_ds = SoundDS(df, data_path='/Users/Saeedeh/Downloads/')
num_items = len(sound_ds)
print(f"There are {num_items} samples in the dataset.")

# Random split of 80:20 between training and validation
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(sound_ds, [num_train, num_val])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

There are 9854 samples in the dataset.


In [ ]:

class AudioClassifier (nn.Module):
  
    def __init__(self):
        super().__init__()
        conv_layers = []

        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=69, out_features=7)

        self.conv = nn.Sequential(*conv_layers)
 
    
    def forward(self, x , fluencyId, firstLanguageId, genderId, currentLanguageId, ageRangeId):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)

        x = x.view(x.shape[0], -1)
        fluencyId = fluencyId.view(x.shape[0], 1)
        firstLanguageId = firstLanguageId.view(x.shape[0], 1)
        genderId = genderId.view(x.shape[0], 1)
        currentLanguageId = currentLanguageId.view(x.shape[0], 1)
        ageRangeId = ageRangeId.view(x.shape[0], 1)

        # Linear layer
        x = self.lin(torch.cat((x, fluencyId, firstLanguageId, genderId, currentLanguageId, ageRangeId), 1))
        
        # Final output
        return x

audioModel = AudioClassifier()
device = torch.device("mps")
audioModel = audioModel.to(device)
next(audioModel.parameters()).device

device(type='mps', index=0)

In [ ]:
##### Training
def training(model, train_dl, num_epochs):

  # Loss Function, Optimizer and Scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01,
                                                steps_per_epoch=int(len(train_dl)),
                                                epochs=num_epochs,
                                                anneal_strategy='linear')

  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    for i, data in enumerate(train_dl):
        inputs, labels, fluencyId, firstLanguageId, genderId, currentLanguageId, ageRangeId = data[0].to(device), data[1].to(device), data[2].to(device) , data[3].to(device), data[4].to(device), data[5].to(device), data[6].to(device)
        
        # Normalize the inputs
        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        optimizer.zero_grad()

        outputs = model(inputs, fluencyId, firstLanguageId, genderId, currentLanguageId, ageRangeId)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()

        # Get the predicted class with the highest score
        _, prediction = torch.max(outputs,1)

        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]
    
    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.3f}, Accuracy: {acc:.3f}')

  print('Finished Training')
  
num_epochs=100   

training(audioModel, train_dl, num_epochs)

Epoch: 0, Loss: 1.817, Accuracy: 0.267
Epoch: 1, Loss: 1.632, Accuracy: 0.325
Epoch: 2, Loss: 1.387, Accuracy: 0.434
Epoch: 3, Loss: 1.168, Accuracy: 0.535
Epoch: 4, Loss: 1.004, Accuracy: 0.606
Epoch: 5, Loss: 0.899, Accuracy: 0.646
Epoch: 6, Loss: 0.799, Accuracy: 0.684
Epoch: 7, Loss: 0.736, Accuracy: 0.713
Epoch: 8, Loss: 0.674, Accuracy: 0.738
Epoch: 9, Loss: 0.650, Accuracy: 0.747
Epoch: 10, Loss: 0.587, Accuracy: 0.776
Epoch: 11, Loss: 0.585, Accuracy: 0.778
Epoch: 12, Loss: 0.541, Accuracy: 0.792
Epoch: 13, Loss: 0.524, Accuracy: 0.801
Epoch: 14, Loss: 0.511, Accuracy: 0.811
Epoch: 15, Loss: 0.488, Accuracy: 0.816
Epoch: 16, Loss: 0.547, Accuracy: 0.789
Epoch: 17, Loss: 0.468, Accuracy: 0.828
Epoch: 18, Loss: 0.450, Accuracy: 0.833
Epoch: 19, Loss: 0.449, Accuracy: 0.830
Epoch: 20, Loss: 0.436, Accuracy: 0.837
Epoch: 21, Loss: 0.441, Accuracy: 0.838
Epoch: 22, Loss: 0.418, Accuracy: 0.847
Epoch: 23, Loss: 0.429, Accuracy: 0.842
Epoch: 24, Loss: 0.407, Accuracy: 0.847
Epoch: 25,

In [ ]:
##### Inference

def inference (model, val_dl):
  correct_prediction = 0
  total_prediction = 0

  with torch.no_grad():
    for data in val_dl:
      inputs, labels, fluencyId, firstLanguageId, genderId, currentLanguageId, ageRangeId = data[0].to(device), data[1].to(device), data[2].to(device) , data[3].to(device), data[4].to(device), data[5].to(device), data[6].to(device)

      inputs_m, inputs_s = inputs.mean(), inputs.std()
      inputs = (inputs - inputs_m) / inputs_s

      outputs = model(inputs, fluencyId, firstLanguageId, genderId, currentLanguageId, ageRangeId)
      
      _, prediction = torch.max(outputs,1)

      correct_prediction += (prediction == labels).sum().item()
      total_prediction += prediction.shape[0]
  
  acc = correct_prediction/total_prediction
  print(f'Accuracy: {acc:.3f}, Total items: {total_prediction}')


inference(audioModel, val_dl)

tensor([[  8.4010, -15.7093,  18.2209,   4.9903,   6.3188, -17.5611, -12.8035],
        [ -7.4305,  19.1177, -20.9044,   5.4683,  -1.8128,  -2.7990,  -9.8849],
        [-18.0324, -11.6643,  -9.4357,   7.2031,  13.8628, -17.6671,  -4.2876],
        [ 17.5834,  -1.0493,   2.6434,  -2.3837,  -0.6755,  -4.3139,  -2.9754],
        [ -7.2298,   8.9883,  -8.7520,   1.5515,   5.8121,  -2.7483,  -1.6769],
        [-13.0916,  -4.3556,  -7.3830,   7.7869,  12.4311, -13.0148, -10.9527],
        [ -4.9445, -12.0783, -11.8361,   6.7732,  11.3117, -17.9822,  -1.4981],
        [-13.6359, -10.7300,  -5.5320,   5.1815,   8.4707,   3.9495,  -1.1930],
        [ -3.5203,   8.6123,  -4.6837,   2.5426,  -1.6214,   3.8115,   3.1690],
        [ -2.8255, -10.8045,  -8.2809,   6.2339, -13.6955,  16.9367,  -0.1468],
        [ -6.5171, -14.8444, -17.6353,  -0.3051,  -7.1505,   9.3927,  15.2160],
        [ 21.9653, -19.0998,  -4.8233,   3.7670,  -1.0026,  -3.8016,  -8.1345],
        [-14.3221,  -6.4738,  -7.7768,  

In [ ]:
### Evaluation

class SoundDSP(Dataset):
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 4000
    self.sr = 16000
    self.channel = 1
            

  def __len__(self):
    return len(self.df)    
    

  def __getitem__(self, idx):
  
    audio_file = self.data_path + self.df.loc[idx, 'path']
    
    id = self.df.loc[idx, 'Id']
    fluencyId = self.df.loc[idx, 'fluencyId']
    firstLanguageId = self.df.loc[idx, 'firstLanguageId']
    genderId = self.df.loc[idx, 'genderId']
    currentLanguageId = self.df.loc[idx, 'currentLanguageId']
    ageRangeId = self.df.loc[idx, 'ageRangeId']


    aud = AudioUtil.open(audio_file)
    reaud = AudioUtil.resample(aud, self.sr)
    rechan = AudioUtil.rechannel(reaud, self.channel)
    dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
    sgram_mel = AudioUtil.spectrogram(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    sgram_mfcc = AudioUtil.mfcc(dur_aud, n_mels=64, n_fft=1024, hop_len=None)
    sgram = torch.cat([sgram_mel, sgram_mfcc])

    return sgram, id, fluencyId, firstLanguageId, genderId, currentLanguageId, ageRangeId

In [ ]:
### preparing evalution dataset

df = pd.read_csv('/Users/Saeedeh/Downloads/dsl_data/evaluation.csv')

df['fluency'] = df['Self-reported fluency level ']
df['FirstLanguage'] = df['First Language spoken']
df['CurrentLanguage'] = df['Current language used for work/school']


df['fluencyId'] = pd.factorize(df.fluency)[0]
df['firstLanguageId'] = pd.factorize(df.FirstLanguage)[0]
df['genderId'] = pd.factorize(df.gender)[0]
df['currentLanguageId'] = pd.factorize(df.CurrentLanguage)[0]
df['ageRangeId'] = pd.factorize(df.ageRange)[0]

df['fluencyId'] = 1
df['firstLanguageId'] = 0
df['currentLanguageId'] = 0

for i in tqdm.tqdm(range(0, len(df))):
  if df['gender'][i] == 'male': 
    df['genderId'][i] = 1
  if df['gender'][i] == 'female': 
    df['genderId'][i] = 0
 

ds_predict = SoundDSP(df, data_path='/Users/Saeedeh/Downloads/')
predict_dl = torch.utils.data.DataLoader(ds_predict, batch_size=5, shuffle=False)

In [ ]:
#### Predict

def predict (model, predict_dl):
  df_predict = pd.read_csv('/Users/Saeedeh/Downloads/dsl_data/sample_submission_final.csv')
  with torch.no_grad():
    for data in predict_dl:
      inputs, ids, fluencyId, firstLanguageId, genderId, currentLanguageId, ageRangeId = data[0].to(device), data[1].to(device), data[2].to(device) , data[3].to(device), data[4].to(device), data[5].to(device), data[6].to(device)

      # Normalize the inputs
      inputs_m, inputs_s = inputs.mean(), inputs.std()
      inputs = (inputs - inputs_m) / inputs_s

      outputs = model(inputs, fluencyId, firstLanguageId, genderId, currentLanguageId, ageRangeId)
      
      # Get the predicted class with the highest score
      _, prediction = torch.max(outputs,1)
      
      for i in range(5):
        id = ids[i].item()
        if df_predict['Id'][id]==ids[i].item():        
          if prediction[i].item()==0:
            df_predict['Predicted'][id] = 'change languagenone'
          elif prediction[i].item()==1:
            df_predict['Predicted'][id] = 'activatemusic'
          elif prediction[i].item()==2:
            df_predict['Predicted'][id] = 'deactivatelights'
          elif prediction[i].item()==3:
            df_predict['Predicted'][id] = 'increasevolume'
          elif prediction[i].item()==4:
            df_predict['Predicted'][id] = 'decreasevolume'
          elif prediction[i].item()==5:
            df_predict['Predicted'][id] = 'increaseheat'
          elif prediction[i].item()==6:
            df_predict['Predicted'][id] = 'decreaseheat'
  df_predict.to_csv('/Users/Saeedeh/Downloads/dsl_data/sample_submission_final.csv', sep=',', index=False)
predict(audioModel, predict_dl)